# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 18 2023 12:11PM,261333,10,9392152,Eye Pharma Inc,Released
1,Apr 18 2023 12:11PM,261332,10,9392674,"Senseonics, Incorporated",Released
2,Apr 18 2023 12:05PM,261331,10,Uniderm.4.14.23.2,"Uniderm USA, Inc - Collagenil Line",Released
3,Apr 18 2023 11:57AM,261329,19,60021611,"GUSA Granules USA, Inc.",Released
4,Apr 18 2023 11:56AM,261330,10,SO94605,"Senseonics, Incorporated",Released
5,Apr 18 2023 11:56AM,261330,10,SO94606,"Senseonics, Incorporated",Released
6,Apr 18 2023 11:41AM,261327,19,9387542,MedStone Pharma LLC,Released
7,Apr 18 2023 11:33AM,261325,10,MSP220860,"Methapharm, Inc.",Released
8,Apr 18 2023 11:33AM,261325,10,MSP220861,"Methapharm, Inc.",Released
9,Apr 18 2023 11:33AM,261325,10,MSP220864,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,261329,Released,1
38,261330,Released,2
39,261331,Released,1
40,261332,Released,1
41,261333,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261329,NaN,NaN,1.0
261330,NaN,NaN,2.0
261331,NaN,NaN,1.0
261332,NaN,NaN,1.0
261333,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260872,5.0,35.0,13.0
261225,0.0,13.0,17.0
261231,9.0,1.0,2.0
261234,0.0,0.0,1.0
261238,0.0,15.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260872,5,35,13
261225,0,13,17
261231,9,1,2
261234,0,0,1
261238,0,15,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260872,5,35,13
1,261225,0,13,17
2,261231,9,1,2
3,261234,0,0,1
4,261238,0,15,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260872,5,35,13
1,261225,,13,17
2,261231,9,1,2
3,261234,,,1
4,261238,,15,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 18 2023 12:11PM,261333,10,Eye Pharma Inc
1,Apr 18 2023 12:11PM,261332,10,"Senseonics, Incorporated"
2,Apr 18 2023 12:05PM,261331,10,"Uniderm USA, Inc - Collagenil Line"
3,Apr 18 2023 11:57AM,261329,19,"GUSA Granules USA, Inc."
4,Apr 18 2023 11:56AM,261330,10,"Senseonics, Incorporated"
6,Apr 18 2023 11:41AM,261327,19,MedStone Pharma LLC
7,Apr 18 2023 11:33AM,261325,10,"Methapharm, Inc."
18,Apr 18 2023 11:29AM,261324,10,"Methapharm, Inc."
19,Apr 18 2023 11:22AM,261322,21,"NBTY Global, Inc."
20,Apr 18 2023 11:01AM,261318,10,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 18 2023 12:11PM,261333,10,Eye Pharma Inc,,,1
1,Apr 18 2023 12:11PM,261332,10,"Senseonics, Incorporated",,,1
2,Apr 18 2023 12:05PM,261331,10,"Uniderm USA, Inc - Collagenil Line",,,1
3,Apr 18 2023 11:57AM,261329,19,"GUSA Granules USA, Inc.",,,1
4,Apr 18 2023 11:56AM,261330,10,"Senseonics, Incorporated",,,2
5,Apr 18 2023 11:41AM,261327,19,MedStone Pharma LLC,,,1
6,Apr 18 2023 11:33AM,261325,10,"Methapharm, Inc.",,,11
7,Apr 18 2023 11:29AM,261324,10,"Methapharm, Inc.",,,1
8,Apr 18 2023 11:22AM,261322,21,"NBTY Global, Inc.",,,1
9,Apr 18 2023 11:01AM,261318,10,Hush Hush,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 12:11PM,261333,10,Eye Pharma Inc,1,,
1,Apr 18 2023 12:11PM,261332,10,"Senseonics, Incorporated",1,,
2,Apr 18 2023 12:05PM,261331,10,"Uniderm USA, Inc - Collagenil Line",1,,
3,Apr 18 2023 11:57AM,261329,19,"GUSA Granules USA, Inc.",1,,
4,Apr 18 2023 11:56AM,261330,10,"Senseonics, Incorporated",2,,
5,Apr 18 2023 11:41AM,261327,19,MedStone Pharma LLC,1,,
6,Apr 18 2023 11:33AM,261325,10,"Methapharm, Inc.",11,,
7,Apr 18 2023 11:29AM,261324,10,"Methapharm, Inc.",1,,
8,Apr 18 2023 11:22AM,261322,21,"NBTY Global, Inc.",1,,
9,Apr 18 2023 11:01AM,261318,10,Hush Hush,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 12:11PM,261333,10,Eye Pharma Inc,1,,
1,Apr 18 2023 12:11PM,261332,10,"Senseonics, Incorporated",1,,
2,Apr 18 2023 12:05PM,261331,10,"Uniderm USA, Inc - Collagenil Line",1,,
3,Apr 18 2023 11:57AM,261329,19,"GUSA Granules USA, Inc.",1,,
4,Apr 18 2023 11:56AM,261330,10,"Senseonics, Incorporated",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 12:11PM,261333,10,Eye Pharma Inc,1.0,NaN,NaN
1,Apr 18 2023 12:11PM,261332,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Apr 18 2023 12:05PM,261331,10,"Uniderm USA, Inc - Collagenil Line",1.0,NaN,NaN
3,Apr 18 2023 11:57AM,261329,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
4,Apr 18 2023 11:56AM,261330,10,"Senseonics, Incorporated",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 12:11PM,261333,10,Eye Pharma Inc,1.0,0.0,0.0
1,Apr 18 2023 12:11PM,261332,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Apr 18 2023 12:05PM,261331,10,"Uniderm USA, Inc - Collagenil Line",1.0,0.0,0.0
3,Apr 18 2023 11:57AM,261329,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
4,Apr 18 2023 11:56AM,261330,10,"Senseonics, Incorporated",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3658317,47.0,11.0,0.0
15,261225,17.0,13.0,0.0
16,261279,0.0,1.0,0.0
19,2873917,20.0,24.0,9.0
20,1044735,18.0,35.0,5.0
21,522639,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3658317,47.0,11.0,0.0
1,15,261225,17.0,13.0,0.0
2,16,261279,0.0,1.0,0.0
3,19,2873917,20.0,24.0,9.0
4,20,1044735,18.0,35.0,5.0
5,21,522639,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,47.0,11.0,0.0
1,15,17.0,13.0,0.0
2,16,0.0,1.0,0.0
3,19,20.0,24.0,9.0
4,20,18.0,35.0,5.0
5,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,47.0
1,15,Released,17.0
2,16,Released,0.0
3,19,Released,20.0
4,20,Released,18.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,9.0,5.0,0.0
Executing,11.0,13.0,1.0,24.0,35.0,1.0
Released,47.0,17.0,0.0,20.0,18.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,0.0,0.0,0.0,9.0,5.0,0.0
1,Executing,11.0,13.0,1.0,24.0,35.0,1.0
2,Released,47.0,17.0,0.0,20.0,18.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,0.0,0.0,0.0,9.0,5.0,0.0
1,Executing,11.0,13.0,1.0,24.0,35.0,1.0
2,Released,47.0,17.0,0.0,20.0,18.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()